In [ ]:
%%capture
!pip install -Uqqq spacy datasets tokenizers plotly
!python -m spacy download en_core_web_sm

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import torch
from torch.utils.data import DataLoader
import pandas as pd
import spacy
from tokenizers.models import WordLevel, BPE, WordPiece
import tokenizers
from datasets import load_dataset
import matplotlib.pyplot as plt
import plotly.express as px
import warnings

In [ ]:
warnings.filterwarnings('ignore')

In [ ]:
dataset = load_dataset("emotion", split='train')
class_names = 'anger fear joy love sadness surprise'.split()
class_lookup = {i:c for i, c in enumerate(class_names)}
class_lookup

In [ ]:
sentence1 = 'The quick brown fox jumped over the lazy dog'
sentence2 = 'Deep learning is fun!'
sentence3 = 'deep learning is hard.'

sentences = [sentence1, sentence2, sentence3]

In [ ]:
cv = CountVectorizer()
sentences_cv = cv.fit_transform(sentences)
sentences_cv = pd.DataFrame(sentences_cv.toarray(), columns=cv.get_feature_names())

sentences_cv

In [ ]:
from tokenizers import normalizers

In [ ]:
normalizer = normalizers.Sequence([
    normalizers.NFD(),
    normalizers.Lowercase(),
    normalizers.StripAccents()
])

In [ ]:
normalizer.normalize_str('Höw aRę ŸõŪ dÔįñg?')

In [ ]:
normalized_senences = [normalizer.normalize_str(s) for s in sentences]
normalized_senences

In [ ]:
for s in normalized_senences:
    print(s.split())

In [ ]:
from tokenizers import pre_tokenizers

In [ ]:
pre_tokenizer = pre_tokenizers.Whitespace()
# split our normalized_sentences
split_sentences = [pre_tokenizer.pre_tokenize_str(s) for s in normalized_senences]
split_sentences

In [ ]:
UNK_TOKEN = '[UNK]'
PAD_TOKEN = '[PAD]'

tokenizer = tokenizers.Tokenizer(model=tokenizers.models.WordLevel(unk_token=UNK_TOKEN))

In [ ]:
tokenizer.normalizer = normalizer
tokenizer.pre_tokenizer = pre_tokenizer

In [ ]:
trainer = tokenizers.trainers.WordLevelTrainer(vocab_size=30000, min_frequency=10, show_progress=True, special_tokens=[PAD_TOKEN, UNK_TOKEN])

In [ ]:
def document_iterator(ds):
    for item in ds:
        yield item['text']

one_document = next(document_iterator(dataset))
one_document

In [ ]:
tokenizer.train_from_iterator(document_iterator(dataset), trainer)

print(f"""
Our tokenizer contains {tokenizer.get_vocab_size()} unique tokens.
""")

In [ ]:
for i in range(5):
    print(f'ID: {i}, token: {tokenizer.id_to_token(i)}')

In [ ]:
encoded = tokenizer.encode(one_document)
pd.DataFrame(zip(encoded.tokens, encoded.ids), columns=['token', 'id']).T

In [ ]:
tokenizer.enable_padding(pad_id=tokenizer.token_to_id(PAD_TOKEN), pad_token=PAD_TOKEN)

In [ ]:
def collate_fn(batch):
    texts = [i['text'] for i in batch]
    encoded = tokenizer.encode_batch(texts)
    ids = [t.ids for t in encoded]
    labels = [i['label'] for i in batch]
    return torch.tensor(ids), torch.tensor(labels)

In [ ]:
dl = DataLoader(dataset, batch_size=8, collate_fn=collate_fn)
encoded_texts, labels = next(iter(dl))

In [ ]:
encoded_texts

In [ ]:
# pass the dataset directly to the dataloader without a collate_fn
dl = DataLoader(dataset, batch_size=8)

# tokenize each batch after it's loaded in the training loop
for batch in dl:
    encoded = tokenizer.encode_batch(batch['text'])
    input_ids = torch.tensor([document.ids for document in encoded])
    labels = batch['label']
    # We'll also write the rest of the steps,
    # although we're not actually training a model at the moment.
    # logits = model(ids)
    # loss = loss_fn(logits, labels)
    # loss.backward()
    # opt.step()
    # opt.zero_grad()
    break

In [ ]:
input_ids

In [ ]:
tokenizer.decode_batch(encoded_texts.numpy())